# MAIC Deeq-Q Learning Workshop
## Cartpole DQN

I would highly recommend giving this paper a read, they're the first big application of Deep-Q: https://arxiv.org/pdf/1312.5602.pdf

Last workshop, we looked at Q-Learning, we'll reiterate a couple of points but we'll also be assuming you have all of the prior knowledge from there.<br/><br/>
The difference between Q-Learning and Deep Q-Learning, is that we replace our Q-table with a neural network.<br/>Before, we use a table which did a lookup on a state and a specific action. The way our network will be set up is that we will take in a state, and output a set of Q-values for every possible action that could be made.<br/><br/>
One of the greatest things about Deep-Q is that we can estimate Q values instead of directly memorizing them, this makes Deep-Q great for enviroments with large or continuous state spaces. We will demonstrate this by making a DQN (Deep-Q Network) for the CartPole game<br/><br/>
We'll also need a buffer of what we've seen from the environment in the past in order to train on this, we call this storage the Experience Replay Buffer<br/><br/>

The main training loop for Deep-Q looks like the following psudocode<br/>
![](psudo.png)<br/><br/>
Now lets get into actual code

Uncomment and run the lines in the following cell that you need

In [ ]:
#!pip install gym
#!pip install tqdm
#!pip install numpy
#!pip install matplotlib
#!pip install tensorflow

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import gym

We can start by seeing how the openai gym environment works, as well as how well making random choices will do

In [ ]:
# setting render_mode to human is good when we want to watch the game but bad for training
env = gym.make('CartPole-v1', render_mode='human')

num_episodes = 5
for i in tqdm(range(num_episodes)):
    # an episode is one "game", we reset our environment at the start of each new episode
    state, _ = env.reset()
    episode_reward = 0
    while True:
        # to progress them game, we tell the environment to take an action
        # we get back the next state (state_prime), the reward for that action, and if the game is over or not (done)
        state_prime, reward, done, _, _ = env.step(env.action_space.sample())
        if done:
            break

Those games all ended pretty quickly, let's try and use Deep-Q to last a little longer.<br/><br/>
- We can start by setting up a new environment that is not rendered.<br/>
- Next, we'll gather some details about the environment, like the state space and the number of actions we are able to take<br/>
- Then we set up our Experience Replay buffer. It will need to hold the state, next state, action, reward, and a flag to know if the game ended at this action<br/>
- We'll set up a batch_size for our neural network to use when sampling the replay buffer<br/>
- Finally we'll set up hyperparameters like we saw from tic-tac-toe like gamma, epsilon, and delta

In [ ]:
env = gym.make('CartPole-v1')
state, _ = env.reset()
action_space = env.action_space.n
# Note that our "states" unlike from tic-tac-toe, are continuous values, so they'd be very hard to put into a Q-Table
state_space = env.observation_space._shape[0]

buffer_size = 100000
buffer_idx = 0
# using an actual array so we can go zoom
buffer = np.zeros((buffer_size, 2*state_space+action_space+1+1))

batch_size = 32
gamma = 0.99
epsilon = 1
epsilon_decay = 0.001

We mentioned that we're using a neural network right?<br/>
Cartpole is pretty simple so this simple model will work, for other, more complicated tasks, the model may need to change<br/><br/>
The only important features about the model is that the input and output shape correspond to the state and action space, and the last layer has a linear activation (no activation)

In [ ]:
def make_model(input_shape, output_shape):
    inputs = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(32, activation='selu', kernel_initializer='lecun_normal')(inputs)
    x = tf.keras.layers.Dense(32, activation='selu', kernel_initializer='lecun_normal')(x)
    x = tf.keras.layers.Dense(32, activation='selu', kernel_initializer='lecun_normal')(x)
    x = tf.keras.layers.Dense(32, activation='selu', kernel_initializer='lecun_normal')(x)
    outputs = tf.keras.layers.Dense(output_shape, kernel_initializer='lecun_normal')(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    return model

This will define how we do back prop as it can be different for Deep-Q.
When our model predicts Q-values for a bunch of actions, we only want to train for the actions we actually took, otherwise we don't know what the reward was. <br/>So we need to zero out all the other values. We inject this into our bellman equation by multiplying our networks predictions by the one hot encoding of the action we took.<br/>
We also need to make sure our bellman equation only applies when the game has not ended, so we multiply the discounted rewards estimated by the target network by the inverse of the done flags.

In [ ]:
def train_step(policy_model, target_model, optimizer, buffer, state_space, action_space):
    batch_indicies = np.random.choice(buffer_size, batch_size, replace=False)
    batch = buffer[batch_indicies]
    states = batch[:,:state_space]
    state_primes = batch[:,state_space:state_space*2]
    actions = batch[:,state_space*2:state_space*2+action_space]
    rewards = batch[:,-2]
    dones = np.invert(batch[:,-1].astype(bool))

    loss_fn = tf.keras.losses.MeanSquaredError()

    # First, just get the targets evaluations
    qprimes = target_model(state_primes)
    # Next, multiply them by the one hot actions to zero out all the indecies which we don't want gradients for
    qprimes = np.multiply(qprimes, actions)
    # Multiply them by gamma for a discount
    qprimes *= gamma
    # Multiply them by the done flags so the states after terminal states are zeroed
    qprimes = np.multiply(qprimes, np.vstack([dones, dones]).T)
    # Stack rewards to make shapes match, and add
    targets = np.vstack([rewards, rewards]).T + qprimes    
    
    with tf.GradientTape() as tape:
        qs = policy_model(states)
        qs *= actions
        loss = loss_fn(qs, targets)
    gradients = tape.gradient(loss, policy_model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, policy_model.trainable_weights))

    return policy_model

Now we're at our main training loop<br/><br/>
We'll start by initializing our policy and target models, then copying the target model onto the policy model<br/>
Then we'll start filling our buffer with games played by random actions<br/>
Once the buffer is full, we'll make a training step at the end of each action, and update the target network at the end of each episode<br/>
This method isn't "the way" to do it, but again for something as simple as cartpole it works. A more standard approach would be to do a training step and update the target network every x and y steps

In [ ]:
policy_model = make_model((state_space,), action_space)
target_model = make_model((state_space,), action_space)
target_model.set_weights(policy_model.get_weights())
optimizer = tf.keras.optimizers.Adam()

episode_rewards = []
steps_taken = 0
num_episodes = 10000

buffer_filled = False
num_trained_episodes = 0
for i in tqdm(range(num_episodes)):
    # Reset environment at the start of every episode
    state, _ = env.reset()
    episode_reward = 0
    # Main game loop
    while True:
        # Epsilon greedy policy
        if np.random.rand() > epsilon:
            action_index = np.argmax(policy_model(state[np.newaxis,...]))
        else:
            action_index = np.random.randint(0, 2)
        # update the environment
        state_prime, reward, done, _, _ = env.step(action_index)
        state_prime
        action = np.zeros((action_space))
        action[action_index] = 1

        # update the replay buffer
        buffer[buffer_idx][:state_space] = state
        buffer[buffer_idx][state_space:state_space*2] = state_prime
        buffer[buffer_idx][state_space*2:state_space*2+action_space] = action
        buffer[buffer_idx][-2] = reward
        buffer[buffer_idx][-1] = done

        # if our buffer is filled, do epsilon decay, and train the model
        episode_reward += reward
        buffer_idx += 1
        if buffer_idx == buffer_size:
            buffer_idx = 0
            if not buffer_filled:
                buffer_filled = True
                print('buffer filled')
        if buffer_filled:
            epsilon = max(epsilon_decay, epsilon-epsilon_decay)
            policy_model = train_step(policy_model, target_model, optimizer, buffer, state_space, action_space)
        # if this episode is over, update the target model and start a new game
        if done:
            if buffer_filled:
                num_trained_episodes += 1
                episode_rewards.append(episode_reward)
            target_model.set_weights(policy_model.get_weights())
            break
        else:
            state = state_prime
    # Once the model starts training a little it can take forEVER to run, so we can cut it off early
    if num_trained_episodes > 50:
        break

policy_model.save_weights('./policy_model.hdf5')

Now that we've trained we can do a quick visualization of how the reward of our model increased over time

In [ ]:
aggregate_rewards_every = 1
average_episode_rewards = []
average_epsilons = []
for i in range(0, len(episode_rewards), aggregate_rewards_every):
    average_episode_rewards.append(np.average(episode_rewards[i:i+aggregate_rewards_every]))
fig, ax1 = plt.subplots()
ax1.set_xlabel(f'Episode / {aggregate_rewards_every}')
ax1.set_ylabel('Reward')
ax1.plot(average_episode_rewards)

fig.tight_layout()
plt.show()

And finally we can watch the game again and hopefully see how our model can imporve significantly over random choices

In [ ]:
policy_model = make_model((state_space,), action_space)
policy_model.load_weights('policy_model.hdf5')
# Inference
env = gym.make('CartPole-v1', render_mode='human')
state, _ = env.reset()
state = state.flatten()
episode_reward = 0

for i in range(5):
    state, _ = env.reset()
    state = state.flatten()

    episode_reward = 0
    while True:
        env.render()
        action = np.argmax(policy_model(state[np.newaxis,...]))
        state, reward, done, _, _ = env.step(action)
        state = state.flatten()
        episode_reward += reward
        if done:
            break
        
    print(f'Reward obtained: {episode_reward}')
env.close()

# Challenge Problems
## Easy (1pt)
- Play around with the hyperparameters of the network (layer size, number of layers, buffer size, batch size, etc), and see if you can get a model that trains faster or better
## Medium (2pt)
- In our train_step function, we currently hard code how many times we need to stack certain things like the rewards in order to fit our action space, are there other functions we can use or make to help us <strong>tile</strong> those values dynamically?
## Hard (3pt)
- What if we tried training on a different environment? Like another game from openai gym? Train a new model and show that it can improve at a new game